#Basics of Data Management

First allow for code in R via rpy2.ipython

In [1]:
!pip -q install rpy2
%load_ext rpy2.ipython

Install and Load R packages

In [2]:
%%R
options(repos = c(CRAN = "https://cloud.r-project.org"))

# Install pacman if it is not already installed
if (!"pacman" %in% rownames(installed.packages())) {
  install.packages("pacman")
}

library(pacman)

# Load the packages you want
p_load(tidyverse, janitor, lubridate, readr)

print("R is ready in Colab ✅")

[1] "R is ready in Colab ✅"


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/pacman_0.5.1.tar.gz'
Content type 'application/x-gzip' length 274400 bytes (267 KB)
downloaded 267 KB


The downloaded source packages are in
	‘/tmp/RtmpV34n3b/downloaded_packages’
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependency ‘snakecase’

trying URL 'https://cloud.r-project.org/src/contrib/snakecase_0.11.1.tar.gz'
trying URL 'https://cloud.r-project.org/src/contrib/janitor_2.2.1.tar.gz'

The downloaded source packages are in
	‘/tmp/RtmpV34n3b/downloaded_packages’

janitor installed
